## BA-analysis


**Step 1**. Download the data and prepare it for analysis
Store the data on visits, orders, and expenses in variables. Optimize the data for analysis. Make sure each column contains the correct data type.
File paths:

    /datasets/visits_log_us.csv. 
    /datasets/orders_log_us.csv. 
    /datasets/costs_us.csv. 

**Step 2**. Make reports and calculate metrics:

**Product**
        How many people use it every day, week, and month?
        How many sessions are there per day? (One user might have more than one session.)
        What is the length of each session?
        How often do users come back?<br>        
**Sales**
        When do people start buying? (In KPI analysis, we're usually interested in knowing the time that elapses between registration and conversion — when the user becomes a customer. For example, if registration and the first purchase occur on the same day, the user might fall into category Conversion 0d. If the first purchase happens the next day, it will be Conversion 1d. You can use any approach that lets you compare the conversions of different cohorts, so that you can determine which cohort, or marketing channel, is most effective.)
        How many orders do they make during a given period of time?
        What is the average purchase size?
        How much money do they bring? (LTV)
        <br>
**Marketing**
        How much money was spent? Overall/per source/over time
        How much did customer acquisition from each of the sources cost?
        How worthwhile where the investments? (ROI)

Plot graphs to display how these metrics differ for various devices and ad sources and how they change in time.
<br>
<br>
**Step 3**. Write a conclusion: advise marketing experts how much money to invest and where.
What sources/platforms would you recommend? Back up your choice: what metrics did you focus on? Why? What conclusions did you draw after finding the metric values?
Format: Complete the task in Jupyter Notebook. Enter the code in code cells and text explanations in markdown cells. Apply formatting and headings.
<br>
<br>
**Description of the data**
The visits table (server logs with data on website visits):

   **Uid**— user's unique identifier
   <br>
   **Device** — user's device
   <br>
   **Start Ts** — session start date and time
   <br>
   **End Ts** — session end date and time
   <br>
   **Source Id** — identifier of the ad source the user came from
   <br>

All dates in this table are in YYYY-MM-DD format.
The orders table (data on orders):

   **Uid** — unique identifier of the user making an order
   <br>
   **Buy Ts** — order date and time
   <br>
   **Revenue** — Yandex.Afisha's revenue from the order
   <br>

The costs table (data on marketing expenses):

   **source_id** — ad source identifier
   <br>
   **dt** — date
   <br>
   **costs** — expenses on this ad source on this day
   <br>

How will my project be assessed?
Your project will be assessed on these criteria. Read over them carefully before you start the project.
Here’s what project reviewers look for when assessing your project:

   How you prepare the data for analysis
   <br>
   What graphs you plot for metrics
   <br>
   How you interpret the resulting graphs
   <br>
   How you calculate and interpret each parameter
   <br>
   How you back up your recommendations for marketing experts and what metrics you use
   <br>
   Whether you follow the project structure and keep the code tidy
   <br>
   The conclusions you come to
   <br>
   Whether you leave comments at each step

In [1]:
!pip install sidetable

In [ ]:
!pip install jupyter-dash

## Step 1.
<br>
Downloading the data and preparing it for analysis Store the data on visits, orders, and expenses in variables. Optimizing the data for analysis. 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import sidetable
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
try:
    visits = pd.read_csv('visits_log_us.csv')
    orders = pd.read_csv('orders_log_us.csv')
    costs = pd.read_csv('costs_us.csv')
    
except:
    visits = pd.read_csv('/datasets/visits_log_us.csv')
    orders = pd.read_csv('/datasets/orders_log_us.csv')
    costs = pd.read_csv('/datasets/costs_us.csv')
    


In [ ]:
visits.head()

In [ ]:
orders.head()

In [ ]:
costs.head()

In [ ]:
visits = visits.rename(columns=str.lower)
orders =  orders.rename(columns=str.lower)


Here we converting the column names from **hightcase** to **lowcase** 

In [ ]:
visits.columns = visits.columns.str.replace(' ','_')
orders.columns = orders.columns.str.replace(' ', '_')

 Here we replacing **spaces**  to  (**'_'**)

In [ ]:
visits.info()

In [ ]:
orders.info()

In [ ]:
visits.stb.missing(style=True)

In [ ]:
visits.head(10)

In [ ]:
visits.tail(10)

In [ ]:
visits.info()

In [ ]:
costs

In [ ]:
costs.info()

In [ ]:
costs.info()

In [ ]:
orders.head()

#### Creating additional date parameters that will help us in cohort analysis

In [ ]:
visits['start_ts'] = pd.to_datetime(visits['start_ts'])
visits['end_ts'] = pd.to_datetime(visits['end_ts']) 
orders['buy_ts'] = pd.to_datetime(orders['buy_ts'])
costs['dt'] = pd.to_datetime(costs['dt'])

In [ ]:
  visits.info()  

In [ ]:
visits['session_year']  = visits['start_ts'].astype('datetime64[Y]')
visits['session_month'] = visits['start_ts'].astype('datetime64[M]')
visits['session_week']  = visits['start_ts'].astype('datetime64[W]')
visits['session_date'] = visits['start_ts'].astype('datetime64[D]')
orders['buy_date'] = orders['buy_ts'].astype('datetime64[D]')

In [ ]:
orders.head()

In [ ]:
visits.head()

## Step 2.
<br>
Making reports and calculating metrics:

### 1.Product
- **How many people use it every day, week, and month?**

**DAU** — the number of daily active (unique) users
<br>
**WAU** — the number of weekly active users
<br>
**MAU** — the number of monthly active users

#### Calculating unique users per day:

In [ ]:
dau=visits.groupby(visits['session_date'])['uid'].nunique().reset_index()
dau.head()

#### Adding reference line with average DAU over time

In [ ]:


fig = px.line(dau,x="session_date", y="uid",title='DAU')
#adding reference line with average DAU over time
fig.add_hline(y=dau['uid'].mean(),line_dash="dash", line_color="purple", annotation_text="average DAU",
             annotation_position="top left")
fig.show()

#### Averege number of users per day:

In [ ]:
dau_total = visits.groupby('session_date').agg({'uid': 'nunique'}).mean()
print(int(dau_total))

#### Calculating unique users per week:

In [ ]:
wau = visits.groupby(visits['session_week'])['uid'].nunique().reset_index()
wau.head()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(layout=go.Layout(
        title=go.layout.Title(text="WAU")))
fig.add_trace(go.Scatter(x=wau['session_week'], 
                        y=wau['uid'],
                    mode='lines+markers',
                    name='lines+markers',line = dict(color='purple', width=5)))
#adding reference line with average DAU over time
fig.add_hline(y=wau['uid'].mean(),line_dash="dash", line_color="red", annotation_text="average WAU",
             annotation_position="top left")
fig.show()

#### Averege number of users per week:

In [ ]:
wau_total = visits.groupby(['session_year', 'session_week']).agg({'uid': 'nunique'}).mean()
print(int(wau_total))

#### Calculating unique users per month:

In [ ]:
mau = visits.groupby(visits['session_month'])['uid'].nunique().reset_index()
wau.head()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(layout=go.Layout(
        title=go.layout.Title(text="MAU")))
fig.add_trace(go.Scatter(x=mau['session_month'], 
                        y=mau['uid'],
                    mode='lines+markers',
                    name='lines+markers',line = dict(color='purple', width=5)))
#adding reference line with average DAU over time
fig.add_hline(y=mau['uid'].mean(),line_dash="dash", line_color="red", annotation_text="average MAU",
             annotation_position="top left")
fig.show()

#### Averege number of users per month:

In [ ]:
mau_total = (visits.groupby(['session_year', 'session_month']).agg({'uid': 'nunique'}).mean())
print(int(mau_total))

As we cee from our graphs there are peak 0n **24 0f november**, so we can estimate that was because of **Thanksgiving** activities.

- **How many sessions are there per day? (One user might have more than one session.)**

#### Unique users per day

In [ ]:
dau_new=visits.groupby(['session_date']).agg({'start_ts': 'count','uid':'nunique'}).reset_index()
dau_new['sessions_per_user']=dau_new['start_ts']/dau_new['uid']
dau_new.head()

In [ ]:
#let's plot sessions per user
fig = px.line(dau_new,
              x="session_date", y="sessions_per_user",title='Sessions per user',
               )
fig.show()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=dau_new['session_date'], 
                        y=dau_new['uid'],
                    mode='lines',
                    name='DAU',line = dict(color='yellow', width=2 )))

fig.add_trace(go.Scatter(x=dau_new['session_date'], 
                        y=dau_new['start_ts'], 
                       
                    mode='lines',
                    name='sessions',line = dict(color='black', width=2 )))


fig.update_layout(
    showlegend=True,
    plot_bgcolor="grey",
    margin=dict(t=10,l=10,b=10,r=10)
)


fig.show()

#### As we ce there are ~1.22 session per user on 24 of November and 4042 users 

### Average session length, or ASL

In [ ]:
visits['session_duration_sec'] = (
    visits['end_ts'] - visits['start_ts']
).dt.seconds
visits['session_duration_sec'].hist(bins=100,range=(0,15000) )


In [ ]:
asl = visits['session_duration_sec'].mode()
print('Averege session length:',asl)

- **How often do users come back?**

### Calculating Retention Rates

In [ ]:
min_visit=visits.groupby(['uid'])['session_date'].min().reset_index()

In [ ]:
min_visit.columns = ['uid', 'first_session']

In [ ]:
min_visit.head()

In [ ]:
visits=visits.merge(min_visit, how='inner',on=['uid'])
visits.head()

- **cohort**: monthly cohort when the user registered/had first session, basically turning his date of registration into month to understand what month he registered in, thus which cohort he belongs to.
- **age**: it's the difference between any current session and first_session, we will also calculate it in 'month' to keep all of the units on one scale) Basically age answers the question : "How long did it pass after registration till the user made this action?" 

In [ ]:
visits['cohort'] = visits['first_session'].astype('datetime64[M]')
visits['age'] = ((pd.to_datetime(visits['session_date']) - pd.to_datetime(visits['first_session'])) / np.timedelta64(1,'M'))\
                                                                        .round().astype('int')
visits.head()

In [ ]:
visits[visits.age==3].sample()

In [ ]:
 cohorts = visits.pivot_table(index='cohort',
                  columns='age',
                  values='uid',
                  aggfunc='nunique').fillna(0)

cohorts

In [ ]:
retention=cohorts.iloc[:,0:].div(cohorts[0], axis=0)
retention

In [ ]:
retention.index=retention.index.astype(str)
plt.figure(figsize=(13, 9))
sns.heatmap(retention, annot=True, fmt='.1%', linewidths=1, linecolor='grey',  vmax=0.1, cbar_kws= {'orientation': 'horizontal'} 
            ).set(title = 'Retention Rate')
plt.show()

## Sales 

In [ ]:
visits.head(1)

In [ ]:
orders.head(1)

- **When do people start buying? (In KPI analysis, we're usually interested in knowing the time that elapses between registration and conversion — when the user becomes a customer.**

In [ ]:
first_order=orders.groupby(['uid'])['buy_ts'].min().reset_index()
first_order.columns = ['uid', 'first_order']
first_order.head()

In [ ]:
first_visit=visits.groupby(['uid'])['start_ts'].min().reset_index()

In [ ]:
purchase = first_visit.merge(first_order, how='inner',on=['uid'])
purchase.head()

In [ ]:
purchase['conversion']=((pd.to_datetime(purchase['first_order'])-pd.to_datetime(purchase['start_ts']))/np.timedelta64(1,'D')).astype('int')

In [ ]:
purchase.head()

In [ ]:
print(' The overall conversion is {:.1%}'.format(purchase['uid'].nunique()/visits['uid'].nunique()))

In [ ]:
fig = px.histogram(purchase, x='conversion', nbins=50)
fig.show()

#### 29772 users made the first purchase  between 1 and 8 days

- **How many orders do they make during a given period of time?**

In [ ]:
orders.head()

In [ ]:
#adding order month, first order  month and age_month
orders['order_month']=orders['buy_ts'].astype('datetime64[M]')
orders.head()

In [ ]:
first_order_month=orders.groupby(['uid']).agg({'order_month':'min'}).reset_index()

In [ ]:
first_order_month.columns=['uid', 'first_order_month']
orders=orders.merge(first_order_month, on='uid')

In [ ]:
orders.head()

In [ ]:
orders['age_month']=round((orders['order_month']-orders['first_order_month'])/np.timedelta64(1, 'M'))

In [ ]:
orders.head()

In [ ]:
cohort=orders.groupby(['first_order_month', 'age_month']).agg({'uid':['nunique','count']}).reset_index()
cohort.columns=['first_order_month', 'age_month', 'users', 'orders']
cohort['orders_per_user']=cohort['orders']/cohort['users']
cohort_piv = cohort.pivot_table(index='first_order_month', columns='age_month', values='orders_per_user', aggfunc='mean')

In [ ]:
cohort_piv.round(2).fillna('')

In [ ]:
plt.figure(figsize=(13, 9))
cohort_piv.index=cohort_piv.index.astype(str)

sns.heatmap(cohort_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='Orders/Period')
plt.show()

- **What is the average purchase size?**

In [ ]:
orders.head()

In [ ]:
avg_cohort=orders.groupby(['first_order_month','order_month'])['revenue'].mean().reset_index()
avg_cohort.head()

In [ ]:
avg_cohort['age_month']=round((avg_cohort['order_month']-avg_cohort['first_order_month'])/np.timedelta64(1, 'M'))

In [ ]:
avg_cohort.head()

In [ ]:
avg_cohort_piv=avg_cohort.pivot_table(
    index='first_order_month', 
    columns='age_month', 
    values='revenue', 
    aggfunc='mean'
)

avg_cohort_piv.round(2).fillna('')

In [ ]:
plt.figure(figsize=(13, 9))
avg_cohort_piv.index=avg_cohort_piv.index.astype(str)

sns.heatmap(avg_cohort_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='Average Purchase')
plt.show()

#### LTV (lifetime value) is the customer's «lifetime» value. It's the total amount of money a customer brings to the company on average by making purchases. When calculating LTV, analysts usually consider the gross profit from a customer per period. 

In [ ]:
orders.head()

In [ ]:
cohort_sizes = orders.groupby('first_order_month').agg({'uid': 'nunique'}).reset_index()
cohort_sizes.columns=['first_order_month','cohort_size']
cohort_sizes.head()

In [ ]:
#get the revenue per cohort in each month
ltv_cohort = orders.groupby(['first_order_month','order_month'])['revenue'].sum().reset_index()
ltv_cohort.columns = ['first_order_month','month','revenue']

In [ ]:
ltv_cohort.head()

In [ ]:
ltv_cohort = ltv_cohort.merge(cohort_sizes,on=['first_order_month'])

In [ ]:


#ltv_cohort = ltv_cohort[ltv_cohort['month'] > ltv_cohort['first_order_month']] 
ltv_cohort['age'] = round((ltv_cohort['month']-ltv_cohort['first_order_month'])/np.timedelta64(1, 'M'))
ltv_cohort['ltv'] = ltv_cohort['revenue']/ltv_cohort['cohort_size']
ltv_cohort

In [ ]:
ltv_cohort_piv=ltv_cohort.pivot_table(
    index='first_order_month', 
    columns='age', 
    values='ltv', 
    aggfunc='sum'
).cumsum(axis=1)


In [ ]:
plt.figure(figsize=(13, 9))
ltv_cohort_piv.index=ltv_cohort_piv.index.astype(str)

sns.heatmap(ltv_cohort_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='LTV')
plt.show()

#### We can cee that the June cohort started pretty low but 4 months later it has 7.62 LTV and continiosly growing 
<br>


### 3.Marketing

- **How much money was spent? Overall/per source/over time**

In [ ]:
costs

In [ ]:
costs['costs_month'] = costs['dt'].astype('datetime64[M]')

In [ ]:
costs

In [ ]:
#sorting cost for futher visualization
costs=costs.sort_values(by=['dt','source_id'])
costs.head()

In [ ]:
print('Total marketing cost is {}'.format(costs['costs'].sum()))

In [ ]:
marketing_per_source=costs.groupby(['source_id','costs_month'])['costs'].sum().reset_index()

In [ ]:
marketing_per_source

In [ ]:
plt.figure(figsize=(20, 9))
fig = px.bar(marketing_per_source, x='source_id', y='costs')
fig.update_xaxes(type='category')
fig.show()

In [ ]:
#plotting costs over time
plt.figure(figsize=(15, 10))
fig = px.line(marketing_per_source, x="costs_month", y="costs", color='source_id')
fig.show()

We can cee that we spant the moste of the budget on sourse 3

In [ ]:
plt.figure(figsize=(15, 10))
fig = px.bar(marketing_per_source, x="costs_month", y="costs")
fig.show()

In [ ]:
#plotting costs over time
plt.figure(figsize=(15, 10))
fig = px.bar(marketing_per_source, x="costs_month", y="costs")
fig.show()

 -  **How much did customer acquisition from each of the sources cost?**

#### CAC (customer acquisition cost) is the cost of attracting a customer—the money a new customer costs a company. 

In [ ]:
marketing_by_month=costs.groupby(['costs_month'])['costs'].sum().reset_index()
marketing_by_month

In [ ]:
buyers_per_moth=orders.groupby(['first_order_month'])['uid'].nunique().reset_index()
buyers_per_moth.columns=['costs_month','buyers']
buyers_per_moth

In [ ]:
CAC_per_month=marketing_by_month.merge(buyers_per_moth,how='left',on=['costs_month'])
CAC_per_month['CAC']=CAC_per_month['costs']/CAC_per_month['buyers']
CAC_per_month

In [ ]:
fig = px.line(CAC_per_month, x="costs_month", y="CAC", title='CAC')
fig.show()

#### We can cee that higthest CAC 10.79 wee have in August 

#### CAC per source

In order to calculate CAC per source we need to know the original source of the user. If we check our original dataset we will see that users can come from different sources ( same problem may occur in your project)

We look at orders table because even if the user didn't buy smth, he still came from the  first source 

In [ ]:
visits.groupby(['uid'])['source_id'].nunique().head(10)

In [ ]:
first_source=visits.sort_values('session_date').groupby('uid').first()['source_id'].reset_index()
first_source.columns=['uid','first_source']
first_source.head()

Merging this data back to purchase becaue CAC calculations are based on buyers

In [ ]:
purchase=orders.merge(first_source,on='uid',how='left')
purchase.head()

Let's see how CAC was changing for each source across time. 

In [ ]:
marketing_by_month_source=costs.groupby(['costs_month','source_id'])['costs'].sum().reset_index()
marketing_by_month_source.head()

In [ ]:
buyers_per_moth_source=purchase.groupby(['first_order_month','first_source'])['uid'].nunique().reset_index()
buyers_per_moth_source.columns=['costs_month','source_id','buyers']
buyers_per_moth_source.head()

In [ ]:
#actual cac calculations
CAC_per_month_source=marketing_by_month_source.merge(buyers_per_moth_source,how='inner',on=['source_id','costs_month'])
CAC_per_month_source['CAC']=CAC_per_month_source['costs']/CAC_per_month_source['buyers']
CAC_per_month_source.head()

In [ ]:
#plotting cac dynamics
fig = px.line(CAC_per_month_source, x="costs_month", y="CAC",color='source_id',title='CAC Sources')
fig.show()

#### We can cee that higthest CAC per Source   wee have on source 3 and source 2 

- **How worthwhile where the investments? (ROI)**

Let's do two version together: ROI per cohort and per source

We will start with ROI per cohort.  <div>
**In genereal, ROI=LTV/CAC**
    <div>
We already have calculations on CAC per month and we have info on ltv in ltv_cohort. 

In [ ]:
CAC_per_month_ROI=CAC_per_month[['costs_month','CAC']]
CAC_per_month_ROI.columns=['first_order_month','CAC']
ROI=ltv_cohort.merge(CAC_per_month_ROI,on=['first_order_month'],how='left')
ROI.head()

In [ ]:
ROI['ROI']=ROI['ltv']/ROI['CAC']
roi_piv = ROI.pivot_table(
    index='first_order_month', columns='age', values='ROI', aggfunc='mean'
).cumsum(axis=1).round(2) 

In [ ]:
roi_piv

In [ ]:
plt.figure(figsize=(13, 9))
roi_piv.index=roi_piv.index.astype(str)
sns.heatmap(roi_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='ROI per cohort')
plt.show()

#### As same as LTV we can cee that there are increase of ROI on 4 month of September and Desember cohorts

In [ ]:
ltv_per_source=purchase.groupby(['first_source'])['uid','revenue'].agg({'uid':'nunique','revenue':'sum'}).reset_index()
ltv_per_source.columns=['source_id','buyers','revenue']
ltv_per_source['ltv']=ltv_per_source['revenue']/ltv_per_source['buyers']
ltv_per_source


### ROMI (return on marketing investment)

Let's see if ROI per source will have a different picture

In [ ]:
roi_per_source=marketing_per_source.merge(ltv_per_source,on=['source_id'])
roi_per_source['cac']=roi_per_source['costs']/roi_per_source['buyers']
roi_per_source['romi']=roi_per_source['ltv']/roi_per_source['cac']
roi_per_source

In [ ]:
plt.figure(figsize=(20, 9))
fig = px.bar(roi_per_source, x='source_id', y='romi')
fig.update_xaxes(type='category')
fig.show()

## Step 3. Overall  Conclusion

#### 1.
<br>
Downloading the data and preparing it for analysis Store the data on visits, orders, and expenses in variables. Optimizing the data for analysis.
<br> 
Here we converting the column names from **hightcase** to **lowcase** 
<br>
 Here we replacing **spaces**  to  (**'_'**)
<br>
Creating additional date parameters that will help us in cohort analysis

####  2.



Making reports and calculating metrics:
<br>
**DAU** — the number of daily active (unique) users
<br>
**WAU** — the number of weekly active users
<br>
**MAU** — the number of monthly active users
<br>
**Calculating unique users per day:** 907
<br>
**Calculating unique users per week:** 5621
<br>
**Calculating unique users per month:** 23228
<br>
As we cee from our graphs there are peak on 24 of november, so we can estimate that was because of **Thanksgiving** activities.
<br>
**Unique sessions/users per day:** As we ce there are ~1.22 session per user on 24 of November and 4042 users 
<br>
**Average session length, or ASL:** **60 sec**
<br>
**Calculating Retention Rates:**

####  3.

**Sales**
<br>
**When do people start buying?**  **29772** users made the first purchase  between **1 and 8 days**
<br>
**How many orders do they make during a given period of time?** In **June** cohotrt we can cee that in month **4** there is **4.19** orders per user  and in **10** month it is **4.45** orders per user
<br>
**What is the average purchase size?** In **September** cohort on **4** mont we have largest amount of averege purchases **62.57**
<br>
**LTV (lifetime value) is the customer's «lifetime» value.**
<br>
We can cee that the **June** cohort started pretty low but **4** months later it has **7.62** LTV and continiosly growing.Also we should check out the **September** cohort that from the **4** month has **11.26** LTV and constantly groving to **13.44** 
<br>


####  4.

**Marketing**
<br>
**CAC (customer acquisition cost) is the cost of attracting a customer—the money a new customer costs a company.** We can cee that **higthest CAC 10.79 wee have in August**
<br>
**CAC per source**
<br>
We can cee that higthest CAC per Source  wee have on **source 3** and **source 2**
<br>
**How worthwhile where the investments? (ROI)** As same as LTV we can cee that there are increase of ROI on 4 month of September and June cohorts
<br>
**ROMI (return on marketing investment)** BIGGEST RETURNS WE HAVE FROM SOURSES  **1**,**2**,**5** and **9**

**Recomendations:**
<br> 
**To check out what ivents  causes activity in cohorts of June and September**
<br> 
**We recommend to drop Sourse 3 and to focus on sourse 1,2,5.and 9**